In [36]:
#Import Cell
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

## Fully developed pipe flow
### Laminar
Reyonds number needs to be less than 2100
Entrance Length = 0.06 * Re * D

### Turbulent
Reyonds number needs to be greater than 4000
Entrance Length = 4.4 * (Re)^(1/6) * D

### General
Due to transition the numbers for classifying flow are not exact
A general formula is:
* Entrance Length = 10 D 
However the 10 increases to 20 then 30 for flows 10^4 < RE < 10^5

In [37]:
pipe_d_in = np.array([0.25, 0.375, 0.5, 0.75, 1, 1.25, 1.5, 2])
pipe_d_m = pipe_d_in / 39.37
pipe_A = pipe_d_m**2 * np.pi

max_velocity = 7
max_density = 1.225
max_delta_P = 500

#300K properties
density = 1.1614
dynamic_mu = 184.6 * 10**-7
kinematic_v = 15.89 * 10**-6

#Fan properties
volumetric_flow = 0.258 #m^3/min

In [38]:
pipe_velocities = volumetric_flow / 60 / pipe_A 
Re = pipe_velocities * pipe_d_m / kinematic_v

df = pd.DataFrame({'Pipe ID (in)':pipe_d_in,'Pipe ID (m)':pipe_d_m,'Pipe Area (m^2)':pipe_A, 'Velocity (m/s)':pipe_velocities, 'Reynolds':Re})



In [39]:
df['Velocity Sensor'] = np.where(df['Velocity (m/s)'] < 7, 'True', 'False')

In [40]:
condition = (df['Velocity (m/s)']<max_velocity) & (df['Reynolds']< 2300)
df['Laminar'] = np.where(condition, 'True', 'False')

In [41]:
df['Entrance Length'] = np.where(df['Laminar']=='True', df['Pipe ID (m)'] * 0.05 * df['Reynolds'], 'N/A')

In [50]:
condition = (df['Velocity Sensor']=='True') & (df['Laminar']=='False')
df['Mixed'] = np.where(condition, df['Pipe ID (m)'] * 1.6 * pow(df['Reynolds'], 0.25), 'N/A')

In [53]:
df['Turbulent'] = df['Pipe ID (m)'] * 4.4 * pow(df['Reynolds'],1/6)
df

,Pipe ID (in),Pipe ID (m),Pipe Area (m^2),Velocity (m/s),Reynolds,Velocity Sensor,Laminar,Entrance Length,Mixed,Turbulent
0,0.250,0.006350,0.000127,33.944498,13565.009174,False,False,N/A,N/A,0.136447
1,0.375,0.009525,0.000285,15.086444,9043.339449,False,False,N/A,N/A,0.191296
2,0.500,0.012700,0.000507,8.486125,6782.504587,False,False,N/A,N/A,0.243121
3,0.750,0.019050,0.001140,3.771611,4521.669725,True,False,N/A,0.2499427480893044,0.340851
4,1.000,0.025400,0.002027,2.121531,3391.252293,True,False,N/A,0.3101305811589849,0.433192
5,1.250,0.031750,0.003167,1.357780,2713.001835,True,False,N/A,0.3666292435329855,0.521722
6,1.500,0.038100,0.004560,0.942903,2260.834862,True,True,4.306899026401196,N/A,0.607328
7,2.000,0.050800,0.008107,0.530383,1695.626147,True,True,4.306899026401196,N/A,0.771861


In [43]:
#Finding desired volumetric flow rate through setting entrace length
L_enter = 101.5 / 1000 * pipe_d_m / pipe_d_m

vol = pd.DataFrame({'Entrance Length':L_enter, 'Pipe ID (in)':pipe_d_in ,'Pipe ID (m)':pipe_d_m})

In [44]:
#0.06 is more conservative 
vol['Lam Re'] = vol['Entrance Length'] / vol['Pipe ID (m)'] / 0.06


In [45]:
vol['Velocity Lam'] = vol['Lam Re'] * kinematic_v / vol['Pipe ID (m)']

In [47]:
vol['Turb Re'] = pow((vol['Entrance Length'] / (vol['Pipe ID (m)'] * 4.4)),6)

In [48]:
vol['Velocity Turb'] = vol['Turb Re'] * kinematic_v / vol['Pipe ID (m)']

In [49]:
vol

,Entrance Length,Pipe ID (in),Pipe ID (m),Lam Re,Velocity Lam,Turb Re,Velocity Turb
0,0.1015,0.250,0.006350,266.403667,0.666637,2298.437143,5.751511
1,0.1015,0.375,0.009525,177.602444,0.296283,201.783233,0.336622
2,0.1015,0.500,0.012700,133.201833,0.166659,35.913080,0.044934
3,0.1015,0.750,0.019050,88.801222,0.074071,3.152863,0.002630
4,0.1015,1.000,0.025400,66.600917,0.041665,0.561142,0.000351
5,0.1015,1.250,0.031750,53.280733,0.026665,0.147100,0.000074
6,0.1015,1.500,0.038100,44.400611,0.018518,0.049263,0.000021
7,0.1015,2.000,0.050800,33.300458,0.010416,0.008768,0.000003
